In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_d0dc69483f8a4de1ab6c1df09fbbadbc = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='jq-L-7BvvoaQ2eUnOB2WhhlSTfAITigHiF0vI4CCsWVD',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_d0dc69483f8a4de1ab6c1df09fbbadbc.get_object(Bucket='usecases-donotdelete-pr-x1wobylqzjzfnc',Key='winequality-white.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df= pd.read_csv(body)
df.head()



,fixedacidity,volatileacidity,citricacid,residualsugar,chlorides,freesulfurdioxide,totalsulfurdioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
X=df.iloc[:, :-1].values
y=df.iloc[:, 11].values

In [4]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [14]:
y_pred=classifier.predict([[7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8]])

In [8]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)


In [9]:
cm

array([[  0,   0,   5,   4,   0,   0,   0],
       [  0,  11,  22,  14,   1,   1,   2],
       [  2,  13, 182,  84,  14,   0,   0],
       [  2,   3,  84, 244,  59,  16,   1],
       [  0,   1,  16,  62,  97,   7,   0],
       [  0,   0,   6,  11,   5,  11,   0],
       [  0,   0,   0,   0,   0,   0,   0]])

In [15]:
y_pred

array([6])

In [16]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2019-04-25 05:40:37,797 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.
2019-04-25 05:41:04,142 - watson_machine_learning_client.wml_client_error - WARNING - Deployment creation failed. Error: 402. {"trace":"6202f7692fa99ce25923329d4850a4b0","errors":[{"code":"deployments_plan_limit_reached","message":"Current plan 'lite' only allows 5 deployments"}]}


In [17]:
wml_credentials = {
  "username": "5d84c453-2449-482b-9784-07bfc87097ee",
  "password": "9f6d627a-1f89-47a9-9fa3-15405257458d",
  "instance_id": "8ac3af8b-681b-4024-9166-311eb6f50447",
  "url": "https://us-south.ml.cloud.ibm.com"
}

In [18]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [19]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "Sasidhar Pokuri", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "sasipv28@gmail.com", 
               client.repository.ModelMetaNames.NAME: "Wine quality"
              }

In [20]:
model_artifact =client.repository.store_model(classifier, meta_props=model_props)

In [21]:
published_model_uid = client.repository.get_model_uid(model_artifact)
published_model_uid

'daa60be3-5a86-4c3d-93fc-d8e2fef7edf3'

In [23]:
deployment = client.deployments.create(published_model_uid, name="wine qulaity")



#######################################################################################

Synchronous deployment creation for uid: 'daa60be3-5a86-4c3d-93fc-d8e2fef7edf3' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='1ac4c6b2-0dbf-4f7d-a5c5-0551194b5ba9'
------------------------------------------------------------------------------------------------




In [24]:
scoring_endpoint = client.deployments.get_scoring_url(deployment)
scoring_endpoint

'https://us-south.ml.cloud.ibm.com/v3/wml_instances/8ac3af8b-681b-4024-9166-311eb6f50447/deployments/1ac4c6b2-0dbf-4f7d-a5c5-0551194b5ba9/online'